In [1]:
import torch
from Model.model import ModelSTGCN
import cv2
import moviepy.editor as mpy
import mmcv
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model,vis_pose_result
import numpy as np
import shutil
import os.path as osp
import os
import glob
import torch.nn as nn

c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 0.5
FONTCOLOR = (255, 255, 255)  # BGR, white
MSGCOLOR = (128, 128, 128)  # BGR, gray
THICKNESS = 1
LINETYPE = 1

# Some Function

In [3]:
def extract_frame(video_path):
    dname = 'temp'
    os.makedirs(dname, exist_ok=True)
    frame_tmpl = osp.join(dname, 'img_{:05d}.jpg')
    cap = cv2.VideoCapture(video_path)
    frame_paths = []
    cnt = 0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag:
            frame_path = frame_tmpl.format(cnt + 1)
            frame_paths.append(frame_path)
            frame=cv2.resize(frame,(640,480))
            cv2.imwrite(frame_path, frame)
            cnt += 1
        else: break
    cap.release()
    return frame_paths

# pose_config = 'mmpose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
# pose_checkpoint = 'hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
# ###########
# det_config = 'Pose/yolox_s_8x8_300e_coco.py'
# det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'
###############
pose_config = 'Pose/hrnet_w48_coco_256x192.py'
pose_checkpoint = 'Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
###########
det_config = 'Pose/yolox_s_8x8_300e_coco.py'
det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'
##############
# initialize pose model
# pose_model = init_pose_model(pose_config, pose_checkpoint)
# # initialize detector
# det_model = init_detector(det_config, det_checkpoint)
def detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr=0.8,device='cuda' ):
    model = init_detector(det_config, det_checkpoint, device)
    assert model.CLASSES[0] == 'person', ('We require you to use a detector '
                                          'trained on COCO')
    results = []
    print('Performing Human Detection for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))
    for frame_path in frame_paths:
        result = inference_detector(model, frame_path)
        # We only keep human detections with score larger than det_score_thr
        result = result[0][result[0][:, 4] >= det_score_thr]
        results.append(result)
        prog_bar.update()
    return results

def pose_inference(pose_config,pose_checkpoint, frame_paths,image_shape, det_results, device='cuda'):
    model = init_pose_model(pose_config, pose_checkpoint, device)
    print('Performing Human Pose Estimation for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))

    num_frame = len(det_results)
    num_person = max([len(x) for x in det_results])
    if num_person == 0:
        kp = np.zeros((1, num_frame, 17, 3), dtype=np.float32)
        return kp    
    kp = np.zeros((num_person, num_frame, 17, 3), dtype=np.float32)
    for i, (f, d) in enumerate(zip(frame_paths, det_results)):
        # Align input format
        if len(d) == 0: 
            prog_bar.update()
            continue
        d = [dict(bbox=x) for x in list(d) if x[-1] > 0.5]
        pose = inference_top_down_pose_model(model, f, d, format='xyxy')[0]
        vis_ske = vis_pose_result(model,f,pose,dataset=model.cfg.data.test.type,show=False)
        if cv2.waitKey(20)& 0xFF==ord('q'): break
        cv2.imshow('',vis_ske)
        for j, item in enumerate(pose):
            #kp and score (x,y,score) / (widt,heigt,1)
            normkp = item['keypoints']
            normkp = normalize_kp(normkp,image_shape)
            kp[j, i] = normkp
            # kp[j,i] = item['keypoints']
        prog_bar.update()
    cv2.destroyAllWindows()
    return kp

def pose_extraction(vid,det_config, det_checkpoint,pose_config,pose_checkpoint,label, det_score_thr=0.5,device='cuda'):
    frame_paths = extract_frame(vid)
    det_results = detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr,device)
    image = cv2.imread(frame_paths[0])
    image_shape = (image.shape[1], image.shape[0])
    pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths,image_shape, det_results, device)
    anno = dict()
    anno['kp'] = pose_results[..., :2]
    anno['kp_score'] = pose_results[..., 2]
    anno['frame_dir'] = osp.splitext(osp.basename(vid))[0]
    anno['img_shape'] = image_shape
    anno['original_shape'] = image_shape
    anno['total_frames'] = pose_results.shape[1]
    anno['label'] = label
    # shutil.rmtree(osp.dirname(frame_paths[0]))
    return anno

def normalize_kp(kp,image_shape):
    w,h = image_shape
    kp[:,0] = (kp[:,0]-w/2)/(w/2)
    kp[:,1] = (kp[:,1]-h/2)/(h/2)
    return kp

# Action Regconigtion

In [4]:
def ActionReg(model: nn.Module = None, file = None,
              det_config = det_config , det_checkpoint = det_checkpoint,
              pose_config = pose_config , pose_checkpoint = pose_checkpoint,
              device = 'cuda'):
    assert all(param is not None for param in [model,file,
                                   det_config,det_checkpoint,
                                   pose_config,pose_checkpoint]),"All param must be give in"
    model.to(device)
    labels = []
    frame_paths = extract_frame(file)
    det_results = detection_inference(det_config,det_checkpoint,frame_paths)
    image = cv2.imread(frame_paths[0])
    image_shape = (image.shape[1], image.shape[0])
    leng_frame = len(frame_paths)
    pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths,image_shape, det_results, device)
    # for person in pose_results:
    #     for window in range(0,leng_frame,30):
    #         temp = torch.from_numpy(person[window:window+30]).to(device)
    #         outputs = model(temp)
    #         pred = torch.argmax(outputs,dim=1)
    #         labels.append(pred)
    # for window in range(0,leng_frame,30):
    #     for person in pose_results:
    #         temp1 = torch.from_numpy(person[window:window+30]).to(device)
    #         outputs = model(temp1)
    #         pred = torch.argmax(outputs,1)
    #         labels.append(pred)
    Action_window = np.zeros((pose_results.shape[0],pose_results.shape[1],1),dtype=np.float16)
    for window in range(0,leng_frame,30):
        # Align input format
        # for j, item in enumerate(pose_results):
        #     feature = np.expand_dims(item[window:window+30],0)
        #     # if feature.shape[1] < 30: break
        #     temp=torch.from_numpy(feature).to(device)
        #     outputs = model(temp)
        #     pred = torch.argmax(outputs,1)
        #     Action_window[j][window:window+30]=pred.item()
        feature = np.expand_dims(pose_results[0][window:window+30],0)
        # if feature.shape[1] < 30: break
        temp=torch.from_numpy(feature).to(device)
        outputs = model(temp)
        pred = torch.argmax(outputs,1)
        Action_window[0][window:window+30]=pred.item()
    return Action_window


In [5]:
# import pandas as pds
# datalst = pds.read_pickle('Data/Test1.pkl')

In [6]:
# datalst[-1]

In [7]:
model=ModelSTGCN(3,2)
model.load_state_dict(torch.load('model_best.pth'))
model.eval()
file='VALID/FALL/Data_fall_11.mp4'
action_window = ActionReg(model,file)

load checkpoint from local path: Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 135/135, 10.5 task/s, elapsed: 13s, ETA:     0sload checkpoint from local path: Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 135/135, 4.0 task/s, elapsed: 33s, ETA:     0s

In [8]:
action_window[0]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],